# K En yakın komşu (K Neighbors nearest)

In [4]:
from sklearn.linear_model import Ridge, Lasso, LinearRegression, RidgeCV, LassoCV, ElasticNetCV, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
#Model oluştururulken alınan hataları göstememesi için
from warnings import filterwarnings
filterwarnings("ignore")

In [6]:
#Veri Seti
df = pd.read_csv("beyzbol_veri.csv")
df = df.dropna()
dms = pd.get_dummies(df[["League", "Division", "NewLeague"]])
X_ = df.drop(["Salary", "League", "Division", "NewLeague"], axis=1).astype("float64")
X = pd.concat([X_ , dms[["League_N", "Division_W", "NewLeague_N"]]], axis = 1)
y = df["Salary"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

## KN model oluşturma & Tahmin

In [8]:
#KNeighbors ile model oluşturmak.
kn_model = KNeighborsRegressor().fit(X_train, y_train)  

In [9]:
kn_model.n_neighbors  #otomatik olarak 5 adet komşu sayısı belirtilmiş

5

In [10]:
kn_model.metric  #uzaklık hesaplanmasında kullanılan yöntem.

'minkowski'

In [11]:
#kn model içerisinden elde edilebilecek veriler
dir(kn_model)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_algorithm_metric',
 '_check_n_features',
 '_estimator_type',
 '_fit',
 '_fit_X',
 '_fit_method',
 '_get_param_names',
 '_get_tags',
 '_kneighbors_reduce_func',
 '_more_tags',
 '_pairwise',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_tree',
 '_validate_data',
 '_y',
 'algorithm',
 'effective_metric_',
 'effective_metric_params_',
 'fit',
 'get_params',
 'kneighbors',
 'kneighbors_graph',
 'leaf_size',
 'metric',
 'metric_params',
 'n_features_in_',
 'n_jobs',
 'n_neighbors',
 'n_samples_fit_',
 'p',
 'predict',
 'radius',
 'sc

### Tahmin

In [12]:
y_pred = kn_model.predict(X_test)

In [13]:
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
RMSE #ortalama hata toplamının karekökü

426.6570764525201

### Model Tuning

* **K en yakın komşu seçiminin en optimum değerini bulmak için GridSearchCV kullanılacaktır.**

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
kn_model = KNeighborsRegressor()
#bir model oluşturulur.
kn_neighbors = {"n_neighbors": np.arange(1,30,1)}
# Optimum komşu sayısını bulmak için sözlük yapısı kullanılmıştır. 1 den 30'a kadar 1 artan değer ile
#sözluk içerisindeki "n_neighbors" ismi kn_model içerisindeki komşuyu temsil eden parametre ile aynı olmalıdır.
#DİKKAT  !!!!!!!
kn_model

KNeighborsRegressor()

In [16]:
kn_model_cv = GridSearchCV(kn_model, kn_neighbors, cv = 10).fit(X_train, y_train)
#GridSerachCV ile oluşturulan model içerisinde komşu sayısı için oluştrulan yapılar girilerek fit edilir.

In [17]:
kn_model_cv.best_params_ 
#en iyi komşu parametresi olarak seçilen değer

{'n_neighbors': 8}

In [18]:
#final model oluşturma : Optimum komşu sayısı bulunduktan sonra
kn_model = KNeighborsRegressor(n_neighbors = kn_model_cv.best_params_["n_neighbors"]).fit(X_train, y_train) #optimum komsu degeri alındı
y_pred = kn_model.predict(X_test)
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
RMSE

413.7094731463598